In [1]:
# libraries
from google.colab import drive
import os
import numpy as np

#Problem 1
### To measure the distance between upper and lower upper planes, we need to compute the distance from x0 to one of the plane times 2. 

#### Upper: wTx + b = 1  x+
#### Lower: wTx + b = -1 x-
#### w is the normal vector of the hyperplane
#### w/ ||w|| is the unit normal vector
#### margin = ((x+) - (x-)) w/ ||w||
#### -> ((x+)w - (x-)w)/ ||w||
#### -> (1 - b + 1 + b)/ ||w||
#### -> 2/ ||w||



In [2]:
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
# use svm classification
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.metrics import silhouette_samples, silhouette_score, f1_score, confusion_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [ ]:
# Problem 2
# 2.a import data 
drive.mount('/content/gdrive')
os.chdir('/content/gdrive/MyDrive/Colab Notebooks/hw6')
Xtrain = np.loadtxt('hw6_data_train.csv', delimiter=',')
ytrain = np.loadtxt('hw6_data_train_labels.csv', delimiter=',')
Xtest = np.loadtxt('hw6_data_test.csv', delimiter=',')
ytest = np.loadtxt('hw6_data_test_labels.csv', delimiter=',')
print('Xtrain shape: %s\nytrain shape: %s\nXtest shape: %s\nytest shape: %s'%(Xtrain.shape, ytrain.shape, Xtest.shape, ytest.shape))

In [ ]:
# 2.b kmean and hierarchy clustering: determining parameters by looping
n = 100
k_ss = []
h_ss = []
for i in range(n):
  kmean_model = KMeans(n_clusters = i + 2, random_state=0)
  hier_model = AgglomerativeClustering(n_clusters = i + 2)

  k_labels = kmean_model.fit_predict(Xtrain)
  h_labels = hier_model.fit_predict(Xtrain)

  k_s = silhouette_score(Xtrain, k_labels)
  h_s = silhouette_score(Xtrain, h_labels)

  k_ss.append(k_s)
  h_ss.append(h_s)


In [ ]:
# 2.b DBSCAN: determining parameters by looping
d_ss = []
max_e = 0
max_min_s = 0
max_ds = 0
for e in np.linspace(0.01,1,100):
  for min_s in np.linspace(5, 20):
    dbscan_model = DBSCAN(eps = e, min_samples = min_s)
    d_labels = dbscan_model.fit_predict(Xtrain)
    try:
      d_s = silhouette_score(Xtrain, d_labels)
      if d_s > max_ds:
        max_e = e 
        max_min_s = min_s
        max_ds = d_s
    except ValueError:
         continue

In [ ]:
# 2.b DBSCAN: determining parameters using elbow -> doesn't work 
neighbors = NearestNeighbors(n_neighbors=int(max_min_s))
neighbors_fit = neighbors.fit(Xtrain)
distances, indices = neighbors_fit.kneighbors(Xtrain)
distances = np.sort(distances, axis=0)
distances = distances[:,1]
plt.plot(distances)

In [ ]:
#max_e = 0.04
print("e: %.4f, min_s: %d" %(max_e, max_min_s))
print("best k: %d" %(np.argmax(k_ss) + 2))
print("best # of clusters for hierarchy: %d" %(np.argmax(h_ss) + 2))

In [ ]:
# 2.b Visualize performance in each clustering
fig, ax = plt.subplots(3,2)

fig.set_size_inches(30, 40)
# n_clusters of each model
k = 14
h = 22
e = 0.13
min_s = 5


kmean_model = KMeans(n_clusters = k, random_state=0)
best_k_labels = kmean_model.fit_predict(Xtrain, ytrain)

hier_model = AgglomerativeClustering(n_clusters = h)
best_h_labels = hier_model.fit_predict(Xtrain)

dbscan_model = DBSCAN(eps = e, min_samples = min_s).fit(Xtrain)
best_d_labels = dbscan_model.labels_
d = len(set(best_d_labels)) - (1 if -1 in best_d_labels else 0)

ax[0,0].set_xlim([0, 1])
ax[0,0].set_ylim([0, len(Xtrain) + (k + 1) * 10])

ax[1,0].set_xlim([0, 1])
ax[1,0].set_ylim([0, len(Xtrain) + (h + 1) * 10])

ax[2,0].set_xlim([0, 1])
ax[2,0].set_ylim([1, len(Xtrain) + (d + 1) * 10])

k_sample_silhouette_values = silhouette_samples(Xtrain, best_k_labels)
h_sample_silhouette_values = silhouette_samples(Xtrain, best_h_labels)
d_sample_silhouette_values = silhouette_samples(Xtrain, best_d_labels)

# source cite: https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html
y_lower = 4
for i in range(k):
    clus_i = k_sample_silhouette_values[best_k_labels == i]

    clus_i.sort()

    size_cluster_i = clus_i.shape[0]
    y_upper = y_lower + size_cluster_i

    color = cm.nipy_spectral(float(i) / k)
    ax[0,0].fill_betweenx(
        np.arange(y_lower, y_upper),
        0,
        clus_i,
        facecolor=color,
        edgecolor=color,
        alpha=0.7,
    )
    ax[0,0].text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
    y_lower = y_upper + 10

ax[0,0].axvline(x = silhouette_score(Xtrain, best_k_labels), color = 'r', label = 'k cluster average silhouette score', linestyle = '--')

y_lower = 10
for i in range(h):
    clus_i = h_sample_silhouette_values[best_h_labels == i]

    clus_i.sort()

    size_cluster_i = clus_i.shape[0]
    y_upper = y_lower + size_cluster_i

    color = cm.nipy_spectral(float(i) / h)
    ax[1,0].fill_betweenx(
        np.arange(y_lower, y_upper),
        0,
        clus_i,
        facecolor=color,
        edgecolor=color,
        alpha=0.7,
    )
    #ax[1,0].text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
    y_lower = y_upper + 0.5

ax[1,0].axvline(x = silhouette_score(Xtrain, best_h_labels), color = 'r', label = 'hierarchy average silhouette score', linestyle = '--')

y_lower = 10
for i in range(d):
    clus_i = d_sample_silhouette_values[best_d_labels == i]

    clus_i.sort()

    size_cluster_i = clus_i.shape[0]
    y_upper = y_lower + size_cluster_i

    color = cm.nipy_spectral(float(i) / d)
    ax[2,0].fill_betweenx(
        np.arange(y_lower, y_upper),
        0,
        clus_i,
        facecolor=color,
        edgecolor=color,
        alpha=0.7,
    )
    #ax[2,0].text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
    y_lower = y_upper + 0.5

ax[2,0].axvline(x = silhouette_score(Xtrain, best_d_labels), color = 'r', label = 'hierarchy average silhouette score', linestyle = '--')

colors = cm.nipy_spectral(best_k_labels.astype(float) / k)
ax[0,1].scatter(
    Xtrain[:, 0], Xtrain[:, 1], marker=".", s=30, lw=0, alpha=0.7, c=colors, edgecolor="k"
)

colors = cm.nipy_spectral(best_h_labels.astype(float) / h)
ax[1,1].scatter(
    Xtrain[:, 0], Xtrain[:, 1], marker=".", s=30, lw=0, alpha=0.7, c=colors, edgecolor="k"
)

colors = cm.nipy_spectral(best_h_labels.astype(float) / d)
ax[2,1].scatter(
    Xtrain[:, 0], Xtrain[:, 1], marker=".", s=30, lw=0, alpha=0.7, c=colors, edgecolor="k"
)



In [ ]:
# 2.c linear regression -> socre is low, adding penalities might improve performance
plt.figure(figsize=(8, 6), dpi=80)
reg = LinearRegression().fit(Xtrain, ytrain)
yhat = reg.predict(Xtest)
print('Linear regression R2 score without penalities: ', reg.score(Xtrain, ytrain))
print('Linear regression MSE score without penalities: ', np.sum((ytest - yhat)**2)/(ytest.shape[0]))
# use 0.5 as threshold
y_hat = []
for i in yhat:
  if i < 0.5:
    y_hat.append(0)
  else:
    y_hat.append(1)
print('Linear regression F1 score without penalities: ', f1_score(ytest, y_hat))

tn, fp, fn, tp = confusion_matrix(ytest, y_hat).ravel()
print("Linear regression percision = %f, recall = %f without penalities" %(tp/(tp+fp), tp/(tp+fn)))
print('coeficients: ', reg.coef_)
print('intercept: ', reg.intercept_)
plt.scatter(np.linspace(0,10, 1000), ytest, color="black")
plt.scatter(np.linspace(0,10, 1000), yhat)

In [ ]:
# 2.c determine hyperparameters for elastic net
max_score = 0
aa = 0
ratio = 0
for a in range(100):
  for l in np.linspace(0, 1, 10):
    en_reg = ElasticNet(alpha = a, l1_ratio = l)
    en_reg.fit(Xtrain, ytrain)
    if en_reg.score(Xtrain, ytrain) > max_score:
      aa = a
      ratio = l
      max_score = en_reg.score(Xtest, ytest)

In [ ]:
# 2.c adding elastic net to the linear regression model doesn't improve the performance, since the data is not linear
l1 = aa*ratio
l2 = 0.5*(aa - aa*ratio)

en_reg = ElasticNet(alpha = aa, l1_ratio = ratio)
en_reg.fit(Xtrain, ytrain)
yhat = en_reg.predict(Xtest)
print("l1 = %f, l2 = %f with score %f" %(l1, l2, en_reg.score(Xtrain, ytrain)))
print('Linear regression R2 score with Elastic Net penalities: ', en_reg.score(Xtrain, ytrain))
print('Linear regression MSE score with Elastic Net penalities: ', np.sum((ytest - yhat)**2)/(ytest.shape[0]))
y_hat = []
for i in yhat:
  if i < 0.5:
    y_hat.append(0)
  else:
    y_hat.append(1)
print('Linear regression F1 score with penalities: ', f1_score(ytest, y_hat))

tn, fp, fn, tp = confusion_matrix(ytest, y_hat).ravel()
print("Linear regression percision = %f, recall = %f with penalities" %(tp/(tp+fp), tp/(tp+fn)))

plt.scatter(np.linspace(0,10, 1000), ytest, color="black")
plt.scatter(np.linspace(0,10, 1000), yhat)

In [ ]:
# 2.c SVM model
# determine hyperparameter: kernel
# use linear kernel type
clf = SVC(kernel='linear')
clf.fit(Xtrain, ytrain) 
yhat = clf.predict(Xtest)
print('Linear SVM mean accuracy : ', clf.score(Xtest, ytest))
print('Linear SVM F1 score: ', f1_score(ytest, yhat))
tn, fp, fn, tp = confusion_matrix(ytest, yhat).ravel()
print("Linear SVM percision = %f, recall = %f " %(tp/(tp+fp), tp/(tp+fn)))

# use poly kernel type
clf = SVC(kernel='poly')
clf.fit(Xtrain, ytrain) 
yhat = clf.predict(Xtest)
print('\nPoly SVM mean accuracy : ', clf.score(Xtest, ytest))
yhat = clf.predict(Xtest)
print('Poly SVM F1 score: ', f1_score(ytest, yhat))
tn, fp, fn, tp = confusion_matrix(ytest, yhat).ravel()
print("Poly SVM percision = %f, recall = %f " %(tp/(tp+fp), tp/(tp+fn)))

clf = SVC(kernel='sigmoid')
clf.fit(Xtrain, ytrain) 
yhat = clf.predict(Xtest)
print('\nSigmoid SVM mean accuracy : ', clf.score(Xtest, ytest))
print('Sigmoid SVM F1 score: ', f1_score(ytest, yhat))
tn, fp, fn, tp = confusion_matrix(ytest, yhat).ravel()
print("Sigmoid SVM percision = %f, recall = %f " %(tp/(tp+fp), tp/(tp+fn)))

clf = SVC(kernel='rbf')
clf.fit(Xtrain, ytrain) 
yhat = clf.predict(Xtest)
print('\nrbf SVM mean accuracy : ', clf.score(Xtest, ytest))
print('rbf SVM F1 score: ', f1_score(ytest, yhat))
tn, fp, fn, tp = confusion_matrix(ytest, yhat).ravel()
print("rbf SVM percision = %f, recall = %f " %(tp/(tp+fp), tp/(tp+fn)))
print("rbf kernel has the best performance")

plt.scatter(np.linspace(0,10, 1000), ytest, color="black")
plt.scatter(np.linspace(0,10, 1000), yhat)